In [ ]:
!nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as tutils
from torch.autograd import Variable
from PIL import Image
from tqdm import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt

## NST Model

Оригинальная архитектура сети VGG19 представлена 16 свёрточными слоями в 5 блоках свёртки, после каждого из которых применяется MaxPooling. В то же время Гатис и др. в своей статье отмечают, что использование AvgPooling вместо MaxPooling улучшает градиент и делает результирующие изображения более привлекательными с точки зрения восприятия человеком.

На разных этапах свёртки информация о входном изображении отличается, увеличивается число накопленных признаков благодаря росту количества применённых фильтров. Одновременно с этим уменьшается разрешение самого изображения после очередного применения downsampling-механизма.

Авторы статьи предприняли попытку визуализировать накопленную информацию на разных слоях CNN. Входное изображение было по очереди воссоздано из первого слоя каждого свёрточного блока. В результате удалось выяснить, что реконструированное изображение с первых слоёв первых трёх блоков почти идентично исходному, а далее информация об отдельных пикселях начинает теряться, но при этом сохраняется "высокоуровневая" информация об объектах на изображении, т.е. об их форме, взаимном расположении и т.п.

Кроме того авторы статьи предприняли различные попытки воссоздания стиля изображения. Для сохранения информации о стиле считалась корреляция между всеми выявленными признаками, которые были найдены фильтрами на разных слоях CNN. Далее были использованы 5 различных наборов корреляций, полученных соответственно с первых слоёв следующих свёрточных блоков: 1; 1 и 2; 1-3; 1-4; 1-5. Авторам удалось выяснить, что использование нового дополнительного слоя для реконструкции стиля постепенно увеличивает масштаб отрисовки отдельного признака, при этом информация о взаимном расположении данных признаков постепенно утрачивается.

Таким образом самым рациональным подходом для Style Transfer алгоритма будет использование карты признаков одного из "верхних" слоёв CNN для переноса контента на результирующее изображение, и использование нескольких "глубоких" слоёв CNN для переноса стиля. Авторами статьи были использованы первые слои всех пяти блоков для сохранения стиля и один слой четвёртого блока для переноса контента.

Для повторения некоторых экспериментов, описанных и проведённых Гатисом и др., определим параметризованный конструктор, для возможности создания сетей с различным pooling-типом и параметризованную функцию forward(), чтобы иметь возможность определять выходы каких слоёв мы будем использовать при переносе стиля.

In [ ]:
class VGG_nst(nn.Module):
    def __init__(self, pooling = None):
        super(VGG_nst,self).__init__()

        if pooling != 'avg' and pooling != 'max':
            raise BaseException("Неправильно указан pooling-тип. " +
                                "Допустимые значения: avg, max.")

        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        

        if pooling is 'avg':
            self.pool_1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool_5 = nn.AvgPool2d(kernel_size=2, stride=2)
        else:
            self.pool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool_5 = nn.MaxPool2d(kernel_size=2, stride=2)

        
    def forward(self, x, out_layers):
        out = {}
        out['conv1_1'] = F.relu(self.conv1_1(x))
        out['conv1_2'] = F.relu(self.conv1_2(out['conv1_1']))
        out['pool_1'] = self.pool_1(out['conv1_2'])
        
        out['conv2_1'] = F.relu(self.conv2_1(out['pool_1']))
        out['conv2_2'] = F.relu(self.conv2_2(out['conv2_1']))
        out['pool_2'] = self.pool_2(out['conv2_2'])
        
        out['conv3_1'] = F.relu(self.conv3_1(out['pool_2']))
        out['conv3_2'] = F.relu(self.conv3_2(out['conv3_1']))
        out['conv3_3'] = F.relu(self.conv3_3(out['conv3_2']))
        out['conv3_4'] = F.relu(self.conv3_4(out['conv3_3']))
        out['pool_3'] = self.pool_3(out['conv3_4'])
        
        out['conv4_1'] = F.relu(self.conv4_1(out['pool_3']))
        out['conv4_2'] = F.relu(self.conv4_2(out['conv4_1']))
        out['conv4_3'] = F.relu(self.conv4_3(out['conv4_2']))
        out['conv4_4'] = F.relu(self.conv4_4(out['conv4_3']))
        out['pool_4'] = self.pool_4(out['conv4_4'])
        
        out['conv5_1'] = F.relu(self.conv5_1(out['pool_4']))
        out['conv5_2'] = F.relu(self.conv5_2(out['conv5_1']))
        out['conv5_3'] = F.relu(self.conv5_3(out['conv5_2']))
        out['conv5_4'] = F.relu(self.conv5_4(out['conv5_3']))
        out['pool_5'] = self.pool_5(out['conv5_4'])

        return [out[layer] for layer in out_layers]

In [ ]:
class GramMatrix(nn.Module):
    def forward(self, input):
        b, c, h, w = input.size()
        f = input.view(b, c, h*w) #bxcx(hxw)
        # torch.bmm(batch1, batch2, out=None)
        # batch1 : bxmxp, batch2 : bxpxn -> bxmxn
        G = torch.bmm(f, f.transpose(1, 2)) # f: BxCx(HxW), f.transpose: Bx(HxW)xC -> BxCxC
        return G.div_(h*w)

class StyleLoss(nn.Module):
    def forward(self, input, target):
        GramInput = GramMatrix()(input)
        return nn.MSELoss()(GramInput, target)

## Utils

In [ ]:
def preprocess(img):
    img = transforms.Resize(imsize)(img)
    img = transforms.ToTensor()(img)
    img = transforms.Lambda(lambda x:x[torch.LongTensor([2, 1, 0])])(img) #RGB to BGR
    img = transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961], std=[0.225, 0.224, 0.229])(img) #subracting imagenet mean
    img = transforms.Lambda(lambda x: x.mul_(255))(img)
    return img


def postprocess(img):
    img = transforms.Lambda(lambda x: x.mul_(1./255))(img)
    img = transforms.Normalize(mean=[-0.40760392, -0.45795686, -0.48501961], std=[1,1,1])(img)
    img = transforms.Lambda(lambda x: x[torch.LongTensor([2, 1, 0])])(img) #turn to RGB
    img = img.clamp_(0,1)
    return img


def load_img(path):
    img = Image.open(path)
    img = preprocess(img)
    img = img.unsqueeze(0)
    return img.to(device)


def load_raw_img(path):
    image = Image.open(path)
    image_tensor = transforms.ToTensor()(image)
    return image_tensor.unsqueeze(0)


def get_preview(tensor):
    image_tensor = tensor.cpu().clone()
    image = transforms.ToPILImage()(image_tensor.squeeze(0))
    image = transforms.Resize(imsize)(image)
    image = transforms.CenterCrop(imsize)(image)
    return image


def show_intermediate_results(content, style, output):
    clear_output(wait=True)
    plt.figure(figsize=(18, 6))
    
    plt.subplot(1, 3, 1)
    plt.imshow(get_preview(content))
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
    plt.title("Content Image")

    plt.subplot(1, 3, 2)
    plt.imshow(get_preview(style))
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
    plt.title("Style Image")

    plt.subplot(1, 3, 3)
    plt.imshow(get_preview(output))
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
    plt.title("Output Image")

    plt.show()
    return

## Train

In [ ]:
def run_style_transfer(iter_num, verbose=True):
    if verbose:
        style_prev = load_raw_img(style_path)
        content_prev = load_raw_img(content_path)

    for iteration in tqdm(range(iter_num)):
        def closure():
            optimizer.zero_grad()
            out = vgg(optimImg, loss_layers)
            totalLossList = []
            for i in range(len(out)):
                layer_output = out[i]
                loss_i = losses[i]
                target_i = targets[i]
                totalLossList.append(loss_i(layer_output, target_i) * weights[i])
            totalLoss = sum(totalLossList)
            totalLoss.backward()
            #print('Loss: %f'%(totalLoss.item()))
            return totalLoss
        optimizer.step(closure)

        if iteration % 5 == 0 and verbose:
            int_result = postprocess(optimImg.data[0].cpu().squeeze())
            show_intermediate_results(content_prev, style_prev, int_result)
    out_img = optimImg.data[0].cpu().squeeze()
    res_img = postprocess(out_img)
    return

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
content_layers = ['conv4_2']
style_layers = ['conv1_1','conv2_1','conv3_1','conv4_1','conv5_1']

In [ ]:
content_path = "/content/Tuebingen_Neckarfront.jpg"
style_path = "/content/picasso.jpg"


imsize = 512
styleImg = load_img(style_path)
contentImg = load_img(content_path)

Загружаем веса для VGG19.

In [ ]:
!wget -c --no-check-certificate https://bethgelab.org/media/uploads/pytorch_models/vgg_conv.pth

In [ ]:
vgg_directory = "/content/vgg_conv.pth" #path to pretrained vgg vgg_directory
vgg = VGG_nst(pooling='avg')
vgg.load_state_dict(torch.load(vgg_directory))

for param in vgg.parameters():
    param.requires_grad = False

vgg.cuda(); # Putting model on cuda

In [ ]:
styleTargets = []
for t in vgg(styleImg, style_layers):
    t = t.detach()
    styleTargets.append(GramMatrix()(t))

contentTargets = []
for t in vgg(contentImg, content_layers):
    t = t.detach()
    contentTargets.append(t)

style_Losses = [StyleLoss()] * len(style_layers)
content_Losses = [nn.MSELoss()] * len(content_layers)

# We only need to go through the vgg once to get all style and content losses

losses = style_Losses + content_Losses
targets = styleTargets + contentTargets
loss_layers = style_layers + content_layers
style_weight = 1e+3  # 1000
content_weight = 1   # 5
weights = [style_weight] * len(style_layers) + [content_weight] * len(content_layers)

In [ ]:
optimImg = Variable(contentImg.data.clone(), requires_grad=True)
optimizer = optim.LBFGS([optimImg])

#Shifting everything to cuda
for loss in losses:
    loss = loss.cuda()
optimImg.cuda();

In [ ]:
run_style_transfer(iter_num=50)